In [1]:
import pandas as pd
import cv2
import os
import glob
import copy
# import 
from pathlib import Path

In [37]:
PATH_DIST = 'D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist/'
DEST_PASSABLES = 'D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/passables/'

if not os.path.exists(DEST_PASSABLES):
    os.makedirs(DEST_PASSABLES)

In [3]:
all_csvs = glob.glob(os.path.join(PATH_DIST, '*'))
print(all_csvs)

['D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\\printed', 'D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\\label', 'D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\\monitor', 'D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\\physical']


In [4]:
def getFilename(path):
    name = Path(path).stem
    # print(name)
    return name

In [10]:
for category in all_csvs:
    category_dist = glob.glob(os.path.join(category, '*'))
    print('*'*50)
    for dist_csv in category_dist:
#         print(dist_csv)
        dist_csv_name = getFilename(dist_csv)
        print(dist_csv_name)
        if '_have_hologram' in dist_csv_name:
            df = pd.read_csv(dist_csv)
#             print(df)
            print(len(df))
#     break

**************************************************
printed_have_hologram
70
printed_insuff_hologram
printed_no_hologram
**************************************************
label_have_hologram
217
label_insuff_hologram
label_no_hologram
**************************************************
monitor_have_hologram
193
monitor_insuff_hologram
monitor_no_hologram
**************************************************
physical_have_hologram
389
physical_insuff_hologram
physical_no_hologram


### Threshold Iterator

In [49]:
category_to_ignore = ['label', 'grayscale']
should_be_static = ['monitor', 'physical']
should_be_nonstatic = ['printed']
low = 0.01
high = 0.50
th = low
threshold_recorder = []
while th < high:
    print('---------------TH:{}---------------'.format(th))
    for category in all_csvs:
        print(category) #all csv by category [physical, printed]
        category_name = getFilename(category)
        if category_name in category_to_ignore:
            continue
        category_dist = glob.glob(os.path.join(category, '*'))
        print('*'*100)
        for dist_csv in category_dist:
            #dist csv by have holo, ins holo, no holo
    #         print(dist_csv)
            dist_csv_name = getFilename(dist_csv)
#             print(dist_csv_name)
            passables_list = []
            if '_have_hologram' in dist_csv_name:
                df = pd.read_csv(dist_csv)
                copy_df = copy.deepcopy(df)
                print(df.head())

                for idx, row in df.iterrows():
                    passables = []
                    nonstatic = False
                    for c in range(len(row)):
                        if c == 0:
                            passables.append(row[c])
                        else:
                            val = float(row[c])
                            if val >= th:
                                passables.append(1)
                                is_it_pass += 1
                            else:
                                passables.append(0)

                    checker = passables.count(1)
                    if checker >= 2:
                        nonstatic = True
                        bin_score = 1
                    else:
                        nonstatic = False
                        bin_score = 0
                    passables.append(bin_score)
                    passables_list.append(passables)

#                 print('List {}'.format(passables_list))
                passables_csv = pd.DataFrame(passables_list,columns=['Onboarding ID', 'F1-F2', 'F2-F3', 'F3-F4', 'Non-static'])

                extended_outpath = '{}/{}/'.format(DEST_PASSABLES, th)
                if not os.path.exists(extended_outpath):
                    os.makedirs(extended_outpath)
                passables_csv.to_csv(r'{}/{}_passables_{}.csv'.format(extended_outpath, th, category_name), index=False)

    print('*'*50)

    th += 0.01
    th = round(th, 2)

---------------TH:0.01---------------
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\printed
****************************************************************************************************
                                  Onboarding ID     F1-F2     F2-F3     F3-F4
0  MTdmZmVmMTQ1ZWI0ZmE3ZTdlODZlMWQzMGQwMjdhYjg=  0.392813  0.340303  0.109077
1  MGFlNzVhYjk0N2Q5OTNkODAyNmUzZmU3NDU1Y2FkMTY=  0.104897  0.096916  0.025376
2  MmMxOGUwYzY3NjBmZTY1Y2Y0NjM3ZTc5YWFkZWE1OWY=  0.120214  0.128120  0.040375
3  MWU3ZmZjZmQ1MDc4MDA0NjEwYzI3YzZjMGZhZjk0YmM=  0.000000  0.538959  0.283219
4  NjAwYjg0MjUwZDQ4ZTI5NmY2NDg5ODVhZDM5YjY2MjQ=  0.125986  0.165356  0.080401
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\label
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\monitor
********************************************************************************************

**************************************************
---------------TH:0.05---------------
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\printed
****************************************************************************************************
                                  Onboarding ID     F1-F2     F2-F3     F3-F4
0  MTdmZmVmMTQ1ZWI0ZmE3ZTdlODZlMWQzMGQwMjdhYjg=  0.392813  0.340303  0.109077
1  MGFlNzVhYjk0N2Q5OTNkODAyNmUzZmU3NDU1Y2FkMTY=  0.104897  0.096916  0.025376
2  MmMxOGUwYzY3NjBmZTY1Y2Y0NjM3ZTc5YWFkZWE1OWY=  0.120214  0.128120  0.040375
3  MWU3ZmZjZmQ1MDc4MDA0NjEwYzI3YzZjMGZhZjk0YmM=  0.000000  0.538959  0.283219
4  NjAwYjg0MjUwZDQ4ZTI5NmY2NDg5ODVhZDM5YjY2MjQ=  0.125986  0.165356  0.080401
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\label
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\monitor
*****************************************

**************************************************
---------------TH:0.09---------------
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\printed
****************************************************************************************************
                                  Onboarding ID     F1-F2     F2-F3     F3-F4
0  MTdmZmVmMTQ1ZWI0ZmE3ZTdlODZlMWQzMGQwMjdhYjg=  0.392813  0.340303  0.109077
1  MGFlNzVhYjk0N2Q5OTNkODAyNmUzZmU3NDU1Y2FkMTY=  0.104897  0.096916  0.025376
2  MmMxOGUwYzY3NjBmZTY1Y2Y0NjM3ZTc5YWFkZWE1OWY=  0.120214  0.128120  0.040375
3  MWU3ZmZjZmQ1MDc4MDA0NjEwYzI3YzZjMGZhZjk0YmM=  0.000000  0.538959  0.283219
4  NjAwYjg0MjUwZDQ4ZTI5NmY2NDg5ODVhZDM5YjY2MjQ=  0.125986  0.165356  0.080401
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\label
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\monitor
*****************************************

**************************************************
---------------TH:0.13---------------
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\printed
****************************************************************************************************
                                  Onboarding ID     F1-F2     F2-F3     F3-F4
0  MTdmZmVmMTQ1ZWI0ZmE3ZTdlODZlMWQzMGQwMjdhYjg=  0.392813  0.340303  0.109077
1  MGFlNzVhYjk0N2Q5OTNkODAyNmUzZmU3NDU1Y2FkMTY=  0.104897  0.096916  0.025376
2  MmMxOGUwYzY3NjBmZTY1Y2Y0NjM3ZTc5YWFkZWE1OWY=  0.120214  0.128120  0.040375
3  MWU3ZmZjZmQ1MDc4MDA0NjEwYzI3YzZjMGZhZjk0YmM=  0.000000  0.538959  0.283219
4  NjAwYjg0MjUwZDQ4ZTI5NmY2NDg5ODVhZDM5YjY2MjQ=  0.125986  0.165356  0.080401
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\label
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\monitor
*****************************************

**************************************************
---------------TH:0.17---------------
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\printed
****************************************************************************************************
                                  Onboarding ID     F1-F2     F2-F3     F3-F4
0  MTdmZmVmMTQ1ZWI0ZmE3ZTdlODZlMWQzMGQwMjdhYjg=  0.392813  0.340303  0.109077
1  MGFlNzVhYjk0N2Q5OTNkODAyNmUzZmU3NDU1Y2FkMTY=  0.104897  0.096916  0.025376
2  MmMxOGUwYzY3NjBmZTY1Y2Y0NjM3ZTc5YWFkZWE1OWY=  0.120214  0.128120  0.040375
3  MWU3ZmZjZmQ1MDc4MDA0NjEwYzI3YzZjMGZhZjk0YmM=  0.000000  0.538959  0.283219
4  NjAwYjg0MjUwZDQ4ZTI5NmY2NDg5ODVhZDM5YjY2MjQ=  0.125986  0.165356  0.080401
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\label
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\monitor
*****************************************

...

**************************************************
---------------TH:0.33---------------
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\printed
****************************************************************************************************
                                  Onboarding ID     F1-F2     F2-F3     F3-F4
0  MTdmZmVmMTQ1ZWI0ZmE3ZTdlODZlMWQzMGQwMjdhYjg=  0.392813  0.340303  0.109077
1  MGFlNzVhYjk0N2Q5OTNkODAyNmUzZmU3NDU1Y2FkMTY=  0.104897  0.096916  0.025376
2  MmMxOGUwYzY3NjBmZTY1Y2Y0NjM3ZTc5YWFkZWE1OWY=  0.120214  0.128120  0.040375
3  MWU3ZmZjZmQ1MDc4MDA0NjEwYzI3YzZjMGZhZjk0YmM=  0.000000  0.538959  0.283219
4  NjAwYjg0MjUwZDQ4ZTI5NmY2NDg5ODVhZDM5YjY2MjQ=  0.125986  0.165356  0.080401
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\label
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\monitor
*****************************************

**************************************************
---------------TH:0.37---------------
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\printed
****************************************************************************************************
                                  Onboarding ID     F1-F2     F2-F3     F3-F4
0  MTdmZmVmMTQ1ZWI0ZmE3ZTdlODZlMWQzMGQwMjdhYjg=  0.392813  0.340303  0.109077
1  MGFlNzVhYjk0N2Q5OTNkODAyNmUzZmU3NDU1Y2FkMTY=  0.104897  0.096916  0.025376
2  MmMxOGUwYzY3NjBmZTY1Y2Y0NjM3ZTc5YWFkZWE1OWY=  0.120214  0.128120  0.040375
3  MWU3ZmZjZmQ1MDc4MDA0NjEwYzI3YzZjMGZhZjk0YmM=  0.000000  0.538959  0.283219
4  NjAwYjg0MjUwZDQ4ZTI5NmY2NDg5ODVhZDM5YjY2MjQ=  0.125986  0.165356  0.080401
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\label
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\monitor
*****************************************

**************************************************
---------------TH:0.41---------------
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\printed
****************************************************************************************************
                                  Onboarding ID     F1-F2     F2-F3     F3-F4
0  MTdmZmVmMTQ1ZWI0ZmE3ZTdlODZlMWQzMGQwMjdhYjg=  0.392813  0.340303  0.109077
1  MGFlNzVhYjk0N2Q5OTNkODAyNmUzZmU3NDU1Y2FkMTY=  0.104897  0.096916  0.025376
2  MmMxOGUwYzY3NjBmZTY1Y2Y0NjM3ZTc5YWFkZWE1OWY=  0.120214  0.128120  0.040375
3  MWU3ZmZjZmQ1MDc4MDA0NjEwYzI3YzZjMGZhZjk0YmM=  0.000000  0.538959  0.283219
4  NjAwYjg0MjUwZDQ4ZTI5NmY2NDg5ODVhZDM5YjY2MjQ=  0.125986  0.165356  0.080401
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\label
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\monitor
*****************************************

**************************************************
---------------TH:0.45---------------
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\printed
****************************************************************************************************
                                  Onboarding ID     F1-F2     F2-F3     F3-F4
0  MTdmZmVmMTQ1ZWI0ZmE3ZTdlODZlMWQzMGQwMjdhYjg=  0.392813  0.340303  0.109077
1  MGFlNzVhYjk0N2Q5OTNkODAyNmUzZmU3NDU1Y2FkMTY=  0.104897  0.096916  0.025376
2  MmMxOGUwYzY3NjBmZTY1Y2Y0NjM3ZTc5YWFkZWE1OWY=  0.120214  0.128120  0.040375
3  MWU3ZmZjZmQ1MDc4MDA0NjEwYzI3YzZjMGZhZjk0YmM=  0.000000  0.538959  0.283219
4  NjAwYjg0MjUwZDQ4ZTI5NmY2NDg5ODVhZDM5YjY2MjQ=  0.125986  0.165356  0.080401
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\label
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\monitor
*****************************************

**************************************************
---------------TH:0.49---------------
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\printed
****************************************************************************************************
                                  Onboarding ID     F1-F2     F2-F3     F3-F4
0  MTdmZmVmMTQ1ZWI0ZmE3ZTdlODZlMWQzMGQwMjdhYjg=  0.392813  0.340303  0.109077
1  MGFlNzVhYjk0N2Q5OTNkODAyNmUzZmU3NDU1Y2FkMTY=  0.104897  0.096916  0.025376
2  MmMxOGUwYzY3NjBmZTY1Y2Y0NjM3ZTc5YWFkZWE1OWY=  0.120214  0.128120  0.040375
3  MWU3ZmZjZmQ1MDc4MDA0NjEwYzI3YzZjMGZhZjk0YmM=  0.000000  0.538959  0.283219
4  NjAwYjg0MjUwZDQ4ZTI5NmY2NDg5ODVhZDM5YjY2MjQ=  0.125986  0.165356  0.080401
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\label
D:/atiqshariff/xendity/projects/fserver_multiF/train_val_cropped_benchmarking_rerun/out/dist\monitor
*****************************************